* 과목: ????
* 이름: ㅇㅇㅇ
* 학번: ??????

과제 제출 전에 위 내용을 작성해 주세요.

# 람다식을 데이타로 만들어 다루어 보자

HelloHaskell 노트북에서처럼 하스켈에서 람다식 문법으로 하스켈을 람다식 계산기처럼 활용해 볼 수도 있다.
하지만 그렇게만 해서는 하스켈이 람다식을 계산하는 방법 이외에 일반적인 성질을 공부할 수 없고
하스켈에서 기본적으로 제공하는 방법 외에 다른 방식으로 람다식을 계산하지 못한다.
람다식도 작은 프로그래밍 언어로 볼 수 있으므로 람다식의 문법 구조를 데이타로 다루면서
람다식의 성질에 대해 생각해 보자.

In [14]:
p = (1,2)
(x,y) = p

p
x
y

(1,2)

1

2

In [16]:
p@(x,y) = (1,2)

p
x
y

(1,2)

1

2

In [17]:
l@(x:xs) = [1,2,3,4]

l = [1,2,3,4]
x:xs = l

l
x
xs

[1,2,3,4]

1

[2,3,4]

In [18]:
-- 하스켈 라이브러리의 합성함수 연산자

h3 = (+5) . (3*)

h3 4

17

In [14]:
compose = \ f -> \ g -> \ x -> f (g x)

-- compose f g x = f (g x)


f1 x = x + 5
f2 x = 3 * x

h1 = compose f1 f2

h1 4

g1 (a, b) = (b, a)
g2 x = (x, x+1)

g1 (g2 3)

h2 = compose g1 g2

h2 3

17

(4,3)

(4,3)

In [2]:
:type compose

compose :: forall t1 t2 t3. (t1 -> t2) -> (t3 -> t1) -> t3 -> t2

In [2]:
-- 변수 이름은 문자열 나타낸다
type Nm = String

-- 람다식 문법 구조
data Tm = Var Nm       -- x
        | App Tm Tm    -- t1 t2
        | Lam Nm Tm    -- 람다 x . t
        deriving (Show, Eq)

-- 람다식을 보기좋게 문자열로 변환해주는 함수
ppTm (Var x) = x
ppTm (Lam x t) = "\\" ++ x ++ " -> " ++ ppTm t
ppTm (App t s) = ppt t ++ " " ++ pps s
  where
  ppt t@(Lam{}) = paren (ppTm t)
  ppt t         = ppTm t
  pps s@(Var{}) = ppTm s
  pps s         = paren (ppTm s)

paren s = "(" ++ s ++ ")"
brack s = "[" ++ s ++ "]"

In [9]:
idTm = Lam "x" (Var "x")
ttTm = Lam "x" (Lam "y" (Var "x")) 
ffTm = Lam "x" (Lam "y" (Var "y")) 

putStr (ppTm (Var "x"))
putStr (ppTm idTm)
putStr (ppTm (App idTm idTm))
putStr (ppTm (App (App ttTm (Var "u")) (Var "w")))

ppTm (App idTm idTm)
:type putStr

x

\x -> x

(\x -> x) (\x -> x)

(\x -> \y -> x) u w

"(\\x -> x) (\\x -> x)"

putStr :: String -> IO ()

In [5]:
-- 하스켈에서 그대로 붙여넣으면 실행도 어느 정도 가능하게
let z = "z" in (\x -> x) ((\x -> x) z)

Line 1: Use id
Found:
\ x -> x
Why not:
idLine 1: Use id
Found:
\ x -> x
Why not:
id

"z"

In [19]:
:ext DeriveFunctor
-- Tm의 구조에서 딱 한군데를 지정해 가리키기 위한 구조(를 만들기 위한 좀 일반적인 구조)
data TmF tm = Here tm -- 여기야 여기
            | Var' Nm | Lam' Nm (TmF tm) | App' (TmF tm) (TmF tm)
  deriving (Show, Functor)

-- 아래 Tm'은 바로 위의 일반적인 구조 TmF의 타입 인자를 Tm으로 정하여
-- Tm의 구조에서 딱 한군데를 지정해 가리키기 위한 구조를 나타낸다
type Tm' = TmF Tm

{-
-- Functor 클래스를 빼고 생각하면 대략 다음과 같다고 보면 된다

data Tm' = Here Tm -- 여기야 여기
         | Var' Nm | Lam' Nm Tm' | App' Tm' Tm'
  deriving Show

-}


-- 보기좋게 문자열로 변환해주는 함수
ppTm' (Here t) = brack (ppTm t)
ppTm' (Var' x) = x
ppTm' (Lam' x t) = "\\" ++ x ++ " -> " ++ ppTm' t
ppTm' (App' t s) = ppt t ++ " " ++ pps s
  where
  ppt t@(Lam'{})       = paren (ppTm' t)
  ppt t@(Here (Lam{})) = ppTm' t
  ppt t                = ppTm' t
  pps s@(Var'{})       = ppTm' s
  pps s@(Here (Var{})) = ppTm' s
  pps s                = paren (ppTm' s)

-- Tm을 Tm'으로 변환
tm2tm' (Var x) = Var' x
tm2tm' (Lam x t) = Lam' x (tm2tm' t)
tm2tm' (App t s) = App' (tm2tm' t) (tm2tm' s)

-- Tm'을 Tm으로 변환 (Here 정보는 사라진다)
tm'2tm (Here t) = t
tm'2tm (Var' x) = Var x
tm'2tm (Lam' x t) = Lam x (tm'2tm t)
tm'2tm (App' t s) = App (tm'2tm t) (tm'2tm s) 

In [21]:
idTm' = tm2tm' idTm

Here(App idTm idTm)
putStr . ppTm' $ Here(App idTm idTm)

App' idTm' (Here(App idTm idTm))
putStr . ppTm' $ App' idTm' (Here(App idTm idTm))

Here (App (Lam "x" (Var "x")) (Lam "x" (Var "x")))

[(\x -> x) (\x -> x)]

App' (Lam' "x" (Var' "x")) (Here (App (Lam "x" (Var "x")) (Lam "x" (Var "x"))))

(\x -> x) ([(\x -> x) (\x -> x)])

In [23]:
-- 아래 람다식의 beta redex를 모두 찾아 각각의 경우를 하나씩 Here로 표시해 보자
putStr . ppTm $ Lam "y" (App idTm (App idTm (Var "y")))

\y -> (\x -> x) ((\x -> x) y)

In [24]:
-- 다음의 두 가지 경우가 있다
putStr . ppTm' $ Lam' "y" (Here(App idTm (App idTm (Var "y"))))
putStr . ppTm' $ Lam' "y" (App' idTm' (Here(App idTm (Var "y"))))

\y -> [(\x -> x) ((\x -> x) y)]

\y -> (\x -> x) ([(\x -> x) y])

In [27]:
-- 아래 람다식에는 beta redex가 3개 있는데 모두 찾아 각각을 Here로 표시해 보라
putStr . ppTm $ App idTm (App idTm idTm `App` App idTm idTm)

(\x -> x) ((\x -> x) (\x -> x) ((\x -> x) (\x -> x)))

In [36]:
-- 여기에 해보세요

[(\x -> x) ((\x -> x) (\x -> x) ((\x -> x) (\x -> x)))]

(\x -> x) ([(\x -> x) (\x -> x)] ((\x -> x) (\x -> x)))

(\x -> x) ((\x -> x) (\x -> x) ([(\x -> x) (\x -> x)]))

In [37]:
-- beta redex를 모두 찾아 리스트로 나열하는 함수를 작성하라
-- beta redex가 없을 경우는 빈 리스트가 결과값이 된다
redexes :: Tm -> [Tm']
redexes (Var x) = []
redexes (Lam x t) = [Lam' x t' | t' <- redexes t]
redexes (App t s) = here
                 ++ [App' t' (tm2tm' s) | t' <- redexes t]
                 ++ [App' (tm2tm' t) s' | s' <- redexes s]
  where
  here = case t of Lam{} -> [Here (App t s)]
                   _     -> []

In [38]:
-- 좀더 예쁘게 유니코드의 그리스 문자 람다 등과 색상 등을 이용해 보여주는 함수들
import IHaskell.Display

dispTm t = Display [html("<code>"++dpTm t++"</code>")]
dispTm' t = Display [html("<code>"++dpTm' t++"</code>")]

dpTm (Var x) = x
dpTm (Lam x t) = "λ" ++ x ++ "." ++ dpTm t
dpTm (App t s) = dpt t ++ " " ++ dps s
  where
  dpt t@(Lam{}) = paren (dpTm t)
  dpt t         = dpTm t
  dps s@(Var{}) = dpTm s
  dps s         = paren (dpTm s)
  
dpTm' (Here t) = "<span style='color:red'>" ++ dpTm t ++ "</span>"
dpTm' (Var' x) = x
dpTm' (Lam' x t) = "λ" ++ x ++ "." ++ dpTm' t
dpTm' (App' t s) = dpt t ++ " " ++ dps s
  where
  dpt t@(Lam'{})      = paren (dpTm' t)
  dpt t@(Here(Lam{})) = paren (dpTm' t)
  dpt t               = dpTm' t
  dps s@(Var'{})      = dpTm' s
  dps s@(Here(Var{})) = dpTm' s
  dps s               = paren (dpTm' s)

dispTm $ Lam "y" (App idTm (App idTm (Var "y")))

dispTm' $ Lam' "y" (Here(App idTm (App idTm (Var "y"))))

dispTm' $ Lam' "y" (App' idTm' (Here(App idTm (Var "y"))))

λy.(λx.x) ((λx.x) y)

λy. (λx.x) ((λx.x) y)

λy.(λx.x) ( (λx.x) y )

In [12]:
putStrLn $ dpTm $ Lam "y" (App idTm (App idTm (Var "y")))

λy.(λx.x) ((λx.x) y)

In [13]:
redexes $ Lam "y" (App idTm (App idTm (Var "y")))

mapM_ (putStrLn . ppTm') . redexes $ Lam "y" (App idTm (App idTm (Var "y")))

map dispTm' . redexes  $ Lam "y" (App idTm (App idTm (Var "y")))

[Lam' "y" (Here (App (Lam "x" (Var "x")) (App (Lam "x" (Var "x")) (Var "y")))),Lam' "y" (App' (Lam' "x" (Var' "x")) (Here (App (Lam "x" (Var "x")) (Var "y"))))]

\y -> [(\x -> x) ((\x -> x) y)]
\y -> (\x -> x) ([(\x -> x) y])

λy. (λx.x) ((λx.x) y)

λy.(λx.x) ( (λx.x) y )

## freshness, alpha-equivalence, capture avoiding substitution
beta reduction을 구현하기 위한 갈길이 멀도다

In [14]:
import Data.List ( (\\), union )

fv (Var x) = [x]
fv (Lam x t) = fv t \\ [x]
fv (App t s) = fv t `union` fv s

In [15]:
dispTm (Lam "x" (App (Var "x") (Var "y")))
fv (Lam "x" (App (Var "x") (Var "y")))

dispTm (App (Lam "x" (App (Var "x") (Var "y"))) (Var "z"))
fv (App (Lam "x" (App (Var "x") (Var "y"))) (Var "z"))

λx.x y

["y"]

(λx.x y) z

["y","z"]

In [16]:
-- 단순무식한 substitition
-- dumbSubst x t s 는 s에서 x가 보이는대로 t로 바꿔친다
-- 이 함수의 문제점은 무엇일까?
dumbSubst :: Nm -> Tm -> Tm -> Tm
dumbSubst x t s@(Var y)   | x == y    = t
                          | otherwise = s
dumbSubst x t (Lam y s1)  = Lam y (dumbSubst x t s1)
dumbSubst x t (App s1 s2) = App (dumbSubst x t s1) (dumbSubst x t s2)

In [17]:
dispTm (Var "x")
dispTm $ dumbSubst "x" (App (Var "f") (Var "z")) (Var "x")

-- 자유(free)이름이 아닌 묶인(bound)이름까지 바꿔친다
dispTm ttTm
dispTm $ dumbSubst "x" (App (Var "f") (Var "z")) ttTm

x

f z

λx.λy.x

λx.λy.f z

In [18]:
-- 자유이름만 바꿔치지만 아직은 엉성한 substitition
-- 이 함수의 문제점은 무엇일까?
naiveSubst :: Nm -> Tm -> Tm -> Tm
naiveSubst x t s@(Var y)    | x == y    = t
                            | otherwise = s
naiveSubst x t s@(Lam y s1) | x == y    = s  -- 묶인이름은 바꿔치지마
                            | otherwise = Lam y (naiveSubst x t s1)
naiveSubst x t (App s1 s2)  = App (naiveSubst x t s1) (naiveSubst x t s2)

In [19]:
-- 묶인이름을 안바꿔치는 건 좋은데
dispTm ttTm
dispTm $ naiveSubst "x" (App (Var "f") (Var "z")) ttTm

-- 바꿔치고 보니 자유이름이 묶여서 갇힌다(capture) ㅠ.ㅠ
dispTm (Lam "z" (App (Var "x") (Var "z")))
dispTm $ naiveSubst "x" (App (Var "f") (Var "z")) (Lam "z" (App (Var "x") (Var "z")))

λx.λy.x

λx.λy.x

λz.x z

λz.f z z

In [20]:
-- 자유이름의 갇힘을 피하는 바꿔치기 capture avoding substutution
-- 우리가 원하는 이 바꿔치기 함수인 subst :: Nm -> Tm -> Tm -> Tm
-- 정의하고자 한다. 근데 이걸 정의하기 위해서는 필요한 개념이 있다.

In [21]:
isFresh :: Nm -> Tm -> Bool
isFresh x = notElem x . fv

In [22]:
dispTm $ Lam "x" (Var "x")
"x" `isFresh` Lam "x" (Var "x")

dispTm $ Lam "y" (Var "x")
"x" `isFresh` Lam "y" (Var "x")

λx.x

True

λy.x

False

In [23]:
{-
fresh x t 는 t에 대해 새로운 이름 즉 t의 자유이름들 중
어떤 것과도 겹치지 않으면서 앞부분이 x로 시작하는 이름을 구한다.
예를 들면 fresh "x" (Var "y") 의 경우라면 이미 "x"가 새롭기 때문에
그냥 "x"를 돌려줘도 무방하다. 하지만 fresh "x" (Var "x")의 경우는
"x"가 (Var "x")에 나타나는 자유이름이므로 "x1"이나 "x'" 혹은 "xx",
"xa", "xabc" 등 겹치지 않는 이름을 아무거나 하나 만들어내면 된다.
이것이 일반적으로 동작하도록 규칙을 나름대로 정해서 만들며 된다.
이런 fresh 함수를 작성해 보라. isFresh 함수를 활용하면 도움이 될 수 있다.
-}
fresh :: Nm -> Tm -> Nm
fresh x t = x -- 이것은 당연히 잘못된 dummy 구현이므로 고쳐야 한다

In [24]:
-- fresh 와 naiveSubst를 이용해 subst를 작성하라
subst :: Nm -> Tm -> Tm -> Tm
subst = naiveSubst -- 이것은 당연히 잘못된 dummy 구현이므로 고쳐야 한다

In [25]:
-- beta t 는 t로부터 가능한 모든 beta-줄이기(reduction) 한 걸음 결과를 리스트로 계산한다.
-- 줄일것(redex)이 없는 경우, 즉 t 가 beta-정규형(normal form)이면 빈 리스트를 돌려준다.
-- redexes 와 subst 를 사용하여 정의하라
beta :: Tm -> [Tm]
beta = map (tm'2tm . fmap betaTop) . redexes

betaTop (App (Lam x t) s) = subst x s t 
betaTop t = error $ "not a top level redex: " ++ ppTm t

In [26]:
mytm = App (App (Lam "u" (Var "u")) (App (Lam "x" (Var "x")) (Lam "y" (Var "y")))) (App (Lam "z" (Var "z")) (Var "w"))

dispTm mytm

(λu.u) ((λx.x) (λy.y)) ((λz.z) w)

In [27]:
map dispTm' $ redexes mytm

(λu.u) ((λx.x) (λy.y)) ((λz.z) w)

(λu.u) ( (λx.x) (λy.y) ) ((λz.z) w)

(λu.u) ((λx.x) (λy.y)) ( (λz.z) w )

In [28]:
map dispTm $ beta mytm

(λx.x) (λy.y) ((λz.z) w)

(λu.u) (λy.y) ((λz.z) w)

(λu.u) ((λx.x) (λy.y)) w

In [29]:
import Data.Tree
-- http://hackage.haskell.org/package/containers/docs/Data-Tree.html
{-
data Tree a = Node a [Tree a]

이걸 이용해서
beta reduction tree 만들어보기
-}
t1 = Node (ppTm mytm) [ Node (ppTm t) [ ] | t <- beta mytm]
putStr $ drawTree t1

(\u -> u) ((\x -> x) (\y -> y)) ((\z -> z) w)
|
+- (\x -> x) (\y -> y) ((\z -> z) w)
|
+- (\u -> u) (\y -> y) ((\z -> z) w)
|
`- (\u -> u) ((\x -> x) (\y -> y)) w

In [30]:
-- 트리가 아닌 일반 리스트로 출력하는 함수
childnode :: [Tm] -> [Tm]
childnode [] = []
childnode (a:as) =  (beta a) ++(childnode as)
-- 위의 함수와 동일하다
chnode :: [Tm] -> [Tm]
chnode = concatMap beta

In [31]:
-- 두 걸음 결과 리스트
map dispTm $ childnode (beta mytm)

(λy.y) ((λz.z) w)

(λx.x) (λy.y) w

(λy.y) ((λz.z) w)

(λu.u) (λy.y) w

(λx.x) (λy.y) w

(λu.u) (λy.y) w

In [32]:
-- 역시 동일..
map dispTm $ chnode (beta mytm)

(λy.y) ((λz.z) w)

(λx.x) (λy.y) w

(λy.y) ((λz.z) w)

(λu.u) (λy.y) w

(λx.x) (λy.y) w

(λu.u) (λy.y) w

In [33]:
-- 세 걸음 결과 리스트, 다음 결과는 모두 w로 동일
map dispTm $ childnode (childnode (beta mytm))

(λz.z) w

(λy.y) w

(λy.y) w

(λz.z) w

(λy.y) w

(λy.y) w

(λy.y) w

(λy.y) w

In [34]:
--루트 노드(람다식) 에서 모든 경우의 경우를 트리로 나타내는 함수
betatree :: Tm -> Tree Tm
{--
cn t = Node t (if null ts then [Node t []| t<-ts] else map cn ts) 
위와 같은 형태는 하스켈에서 잘 사용하지 않는다.
아래와 같이 guard형태로 사용 
--}

-- 계속 진행하다 다음 걸음이 null일 경우 멈춘다
betatree t  | null ts   = Node t [Node t' [] | t'<-ts] 
            | otherwise = Node t (map betatree ts)
  where
  -- 1 step beta result
  ts :: [Tm]
  ts = beta t

In [35]:
mybtree = betatree mytm

putStr $ drawTree (fmap ppTm mybtree)

(\u -> u) ((\x -> x) (\y -> y)) ((\z -> z) w)
|
+- (\x -> x) (\y -> y) ((\z -> z) w)
|  |
|  +- (\y -> y) ((\z -> z) w)
|  |  |
|  |  +- (\z -> z) w
|  |  |  |
|  |  |  `- w
|  |  |
|  |  `- (\y -> y) w
|  |     |
|  |     `- w
|  |
|  `- (\x -> x) (\y -> y) w
|     |
|     `- (\y -> y) w
|        |
|        `- w
|
+- (\u -> u) (\y -> y) ((\z -> z) w)
|  |
|  +- (\y -> y) ((\z -> z) w)
|  |  |
|  |  +- (\z -> z) w
|  |  |  |
|  |  |  `- w
|  |  |
|  |  `- (\y -> y) w
|  |     |
|  |     `- w
|  |
|  `- (\u -> u) (\y -> y) w
|     |
|     `- (\y -> y) w
|        |
|        `- w
|
`- (\u -> u) ((\x -> x) (\y -> y)) w
   |
   +- (\x -> x) (\y -> y) w
   |  |
   |  `- (\y -> y) w
   |     |
   |     `- w
   |
   `- (\u -> u) (\y -> y) w
      |
      `- (\y -> y) w
         |
         `- w

In [36]:
import qualified Data.Char as Char
import Text.Printf
 
encode :: Char -> String
encode c
  | c == ' ' = "%20" 
  | Char.isAlphaNum c || c `elem` "-._~" = [c]  
  | c `elem` "\"" = "%22"
  | otherwise = printf "%%%02X" c
 
urlEncode :: String -> String
urlEncode = concatMap encode

In [37]:
tree = last $ subForest mybtree

-- [rootLabel t | t <- subForest tree]
-- [e | Node e _ <- subForest tree]

tr2e (Node e []) = []  
tr2e (Node e ts) = [(e,e') | Node e' _ <- ts] `union` (foldr1 union $ map tr2e ts)

--mapM_ print $ tr2e tree

tr2v (Node e []) = [e]
tr2v (Node e ts) = [e] `union` (foldr1 union $ map tr2v ts)

labels tree = concat [ show n++" [label=\""++dpTm t++"\"]; " | (t,n)<- zip (tr2v tree) [0..]]

--Display [html $ labels tree]

--lookup (Var "w") (zip (tr2v tree) [0..])


fromJust (Just x) = x

edges tree = concat [show x++" -> "++show y++"; " | (x,y) <-edges]
  where
   edges = [(fromJust $ lookup e tbl, fromJust $ lookup e' tbl) | (e,e') <- tr2e tree]
   tbl = zip (tr2v tree) [0..]

edges tree

tree2graph tree = "digraph G { "++ labels tree ++ edges tree ++" }"

Display [html $ tree2graph tree]

-- dispTree tree = Display [html $ tree2graph tree]

-- dispTree mybtree

displayGraph tree = Display [html ("<img src='" ++ url ++ "'/>")]
  where url = "https://graphviz.glitch.me/graphviz?layout=dot&format=svg&mode=download&graph="
           ++ urlEncode (tree2graph tree)
           

displayGraph mybtree

"0 -> 1; 0 -> 4; 1 -> 2; 2 -> 3; 4 -> 2; "

digraph G { 0 [label="(λu.u) ((λx.x) (λy.y)) w"]; 1 [label="(λx.x) (λy.y) w"]; 2 [label="(λy.y) w"]; 3 [label="w"]; 4 [label="(λu.u) (λy.y) w"]; 0 -> 1; 0 -> 4; 1 -> 2; 2 -> 3; 4 -> 2; }

<img src='https://graphviz.glitch.me/graphviz?layout=dot&format=svg&mode=download&graph=digraph%20G%20%7B%200%20%5Blabel%3D%22%28λu.u%29%20%28%28λx.x%29%20%28λy.y%29%29%20%28%28λz.z%29%20w%29%22%5D%3B%201%20%5Blabel%3D%22%28λx.x%29%20%28λy.y%29%20%28%28λz.z%29%20w%29%22%5D%3B%202%20%5Blabel%3D%22%28λy.y%29%20%28%28λz.z%29%20w%29%22%5D%3B%203%20%5Blabel%3D%22%28λz.z%29%20w%22%5D%3B%204%20%5Blabel%3D%22w%22%5D%3B%205%20%5Blabel%3D%22%28λy.y%29%20w%22%5D%3B%206%20%5Blabel%3D%22%28λx.x%29%20%28λy.y%29%20w%22%5D%3B%207%20%5Blabel%3D%22%28λu.u%29%20%28λy.y%29%20%28%28λz.z%29%20w%29%22%5D%3B%208%20%5Blabel%3D%22%28λu.u%29%20%28λy.y%29%20w%22%5D%3B%209%20%5Blabel%3D%22%28λu.u%29%20%28%28λx.x%29%20%28λy.y%29%29%20w%22%5D%3B%200%20-%3E%201%3B%200%20-%3E%207%3B%200%20-%3E%209%3B%201%20-%3E%202%3B%201%20-%3E%206%3B%202%20-%3E%203%3B%202%20-%3E%205%3B%203%20-%3E%204%3B%205%20-%3E%204%3B%206%20-%3E%205%3B%207%20-%3E%202%3B%207%20-%3E%208%3B%208%20-%3E%205%3B%209%20-%3E%206%3B%209%20-%3E%208%3B%20%20%7D'/>

더 이상 줄일 redex가 없는 식을 normal form이라고 한다. 영어로 normal은 보통 "정상"으로 번역되지만 무엇이 정상인지 알려면 어떤 기준이 있어야 한다. 그런 의미에서 norm이라는 명사형은 "기준"(사회적 용어로는 사회에서 기준이 되는 예의, 규칙, 관습 등의 뜻으로도 쓰임)이라는 뜻이다. 왜 더 이상 줄일 것이 없는 식을 normal form이라 하나면 어떤 식의 특징이나 성질을 대표하는 값이 무엇이가를 생각할 때 그 식을 대표하는 값으로서 더 이상 줄일 것이 없어 계산이 진행되지 않는 형태를 기준으로 삼는 것이 자연스럽기 때문이다.

위 그래프에서 확인할 수 있는 현상을 비유하면 다음과 같이 말할 수 있다.

> 모로 가도 서울로 가게 되어 있다 (단, 삼천포로 빠지지만 않으면)

어떤 redex를 먼저 줄일지 고르는 것에 관계없이 normal form에 도달한다면 항상 똑같은 normal form(즉, 서울)에 도달한다는 것이다.
단, 식의 일부분 중에 redex가 없어지지 않고 무한히 반복되는 계산을 영원히 반복하고 있는 경우라면 (즉, 삼천포로 빠지는 경우라면) normal form에 도달하지 못할 수도 있다.

이것은 위에서 예시로 보여준 경우 뿐 아니라 모든 람다식이 만족하는 일만적인 성질로서 이를 증명한 사람들의 이름을 따서
[처치 로서 정리](https://ko.wikipedia.org/wiki/%EC%B2%98%EC%B9%98-%EB%A1%9C%EC%84%9C_%EC%A0%95%EB%A6%AC)[(Church-Rossor theorem)](https://en.wikipedia.org/wiki/Church%E2%80%93Rosser_theorem)라는 람다계산법의 이론적 성질에 대한 중요한 정리들 중 하나이다.

참고로 삼천포로 빠지는 대표적인 람다식은 다음과 같다.

In [51]:
omega = Lam "x" (App x x) where x = Var "x"

infloop = App omega omega

dispTm omega

dispTm infloop

map dispTm $ beta infloop

λx.x x

(λx.x x) (λx.x x)

(λx.x x) (λx.x x)

# 과제2 (제출기한: 10/8 밤까지, 배점: 3점)
리스트에 적용하는 `take` 함수와 비슷한 계산을 `Data.Tree` 모듈의 트리에 대해 하는 `takeTree`를 작성하라.
`takeTree n tree`는 `tree`를 주어진 높이 `1` 까지만 출력한다.

추가사항: 서로 다른 타입의 값이 들어가는 트리를 구성해 테스트 (2가지 이상)

`Data.Tree` 모듈에 정의된 트리 데이타 타입은 기본적으로 아래와 같다. (레코드 문법을 사용해서 실제 출력해 보면 좀더 복잡하게 보이지만 안에 들어있는 데이타만 보면 똑같은 구조이다.)

```haskell
data Tree a = Node a [Tree a]
```

In [38]:
takeTree :: Int -> Tree a -> Tree a
takeTree n _   | n <= 0 = error (show n++" is not positive")
takeTree 1 (Node x _)  = undefined
takeTree n (Node x ts) = undefined

예컨대, 위에 정의된 `mybtree`에 대해 `takeTree 3 mybtree`를 계산한 결과는 다음과 같아야 한다.

```
(\u -> u) ((\x -> x) (\y -> y)) ((\z -> z) w)
|
+- (\x -> x) (\y -> y) ((\z -> z) w)
|  |
|  +- (\y -> y) ((\z -> z) w)
|  |
|  `- (\x -> x) (\y -> y) w
|
+- (\u -> u) (\y -> y) ((\z -> z) w)
|  |
|  +- (\y -> y) ((\z -> z) w)
|  |
|  `- (\u -> u) (\y -> y) w
|
`- (\u -> u) ((\x -> x) (\y -> y)) w
   |
   +- (\x -> x) (\y -> y) w
   |
   `- (\u -> u) (\y -> y) w
```